# Unsupervised Learning

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt

# StatsModel
from statsmodels. datasets import get_rdataset

# SciPy
from scipy.cluster. hierarchy import (dendrogram, cut_tree)

# Scikit-Learn
from sklearn. decomposition import PCA
from sklearn. preprocessing import StandardScaler
from sklearn.cluster import (KMeans, AgglomerativeClustering)

# ISLP
from ISLP import load_data
from ISLP.cluster import compute_linkage

### Principal Components Analysis

In [ ]:
USArrests = get_rdataset('USArrests').data

USArrests

In [ ]:
USArrests.columns

In [ ]:
USArrests.mean()

In [ ]:
USArrests.var()

In [ ]:
scaler = StandardScaler(with_std=True,
                        with_mean=True)

USArrests_scaled = scaler.fit_transform(USArrests)

In [ ]:
pcaUS = PCA()

In [ ]:
pcaUS.fit(USArrests_scaled)

In [ ]:
pcaUS.mean_

In [ ]:
scores = pcaUS.transform(USArrests_scaled)

In [ ]:
pcaUS.components_

In [ ]:
i, j = 0, 1 # which components
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.scatter(scores[:, 0], scores[:, 1])
ax.set_xlabel('PC%d' % (i+1))
ax.set_ylabel('PC%d' % (j+1))

for k in range(pcaUS. components_.shape[1]):
    ax.arrow (0, 0, pcaUS.components_[i, k], pcaUS.components_[j, k])
    ax.text(pcaUS.components_[i, k],
            pcaUS.components_[j, k],
            USArrests.columns[k])

plt.grid(True)
plt.show()

In [ ]:
scale_arrow = s_ = 2
scores[:, 1] *= -1

pcaUS.components_[1] *= -1 # flip the y-axis
fig, ax = plt. subplots(1, 1, figsize=(8, 8))
ax.scatter(scores[:, 0], scores[:, 1])
ax.set_xlabel('PC%d' % (i+1))
ax.set_ylabel('PC%d' % (j+1))

for k in range(pcaUS.components_.shape[1]):
    ax.arrow(0, 0, s_*pcaUS. components_[i,k], s_*pcaUS.components_[
        j,k])
    ax.text(s_*pcaUS.components_[i, k],
            s_*pcaUS.components_[j, k],
            USArrests.columns[k])

plt.grid(True)
plt.show()

In [ ]:
scores.std(0, ddof =1)

In [ ]:
pcaUS.explained_variance_

In [ ]:
pcaUS.explained_variance_ratio_

In [ ]:
%%capture
fig, axes = plt. subplots(1, 2, figsize=(15, 6))

ticks = np.arange(pcaUS.n_components_)+1
ax = axes[0]
ax.plot(ticks,
        pcaUS.explained_variance_ratio_,
        marker='o')
ax.set_xlabel('Principal Component ');
ax.set_ylabel('Proportion of Variance Explained ')
ax.set_ylim([0 ,1])
ax.set_xticks(ticks)

In [ ]:
ax = axes [1]

ax.plot(ticks,
        pcaUS.explained_variance_ratio_.cumsum(),
        marker='o')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Cumulative Proportion of Variance Explained')
ax.set_ylim([0, 1])
ax.set_xticks(ticks)

# plt.grid(True)
# plt.show()
fig

In [ ]:
a = np.array([1, 2, 8, -3])

np.cumsum(a)

### Matrix Completion

In [ ]:
X = USArrests_scaled

U, D, V = np.linalg.svd(X, full_matrices =False)
U.shape, D.shape, V.shape

In [ ]:
V

In [ ]:
pcaUS.components_

In [ ]:
(U * D[None, :])[:3]

In [ ]:
scores[:3]

In [ ]:
n_omit = 20
np.random.seed(15)

r_idx = np.random.choice(np.arange(X.shape[0]),
                         n_omit,
                         replace=False)

c_idx = np.random.choice(np.arange(X.shape[1]),
                         n_omit,
                         replace=True)

Xna = X.copy()
Xna[r_idx, c_idx] = np.nan

In [ ]:
def low_rank(X, M=1):
    U, D, V = np.linalg.svd(X)
    L = U[:, :M] * D[None, :M]
    return L.dot(V[:M])

In [ ]:
Xhat = Xna.copy()

Xbar = np.nanmean(Xhat, axis =0)

Xhat[r_idx, c_idx] = Xbar[c_idx]

In [ ]:
thresh = 1e-7
rel_err = 1
count = 0

ismiss = np.isnan(Xna)
mssold = np.mean(Xhat[~ismiss]**2)
mss0 = np.mean(Xna[~ismiss]**2)

In [ ]:
while rel_err > thresh:
    count += 1
    # Step 2(a)
    Xapp = low_rank (Xhat, M=1)
    # Step 2(b)
    Xhat[ismiss] = Xapp[ismiss]
    # Step 2(c)
    mss = np.mean (((Xna - Xapp)[~ismiss])**2)
    rel_err = (mssold - mss) / mss0
    mssold = mss
    print("Iteration : {0}, MSS :{1:.3f}, Rel.Err {2:.2e}"
        .format(count, mss, rel_err))

In [ ]:
np. corrcoef(Xapp[ismiss], X[ismiss])[0, 1]

### Clustering

#### K-Means Clustering

In [ ]:
np.random.seed(0);

X = np.random.standard_normal((50, 2));

X[:25, 0] += 3;
X[:25, 1] -= 4;

In [ ]:
kmeans = KMeans(n_clusters=2,
                random_state=2,
                n_init =20).fit(X)

In [ ]:
kmeans.labels_

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.scatter(X[:, 0], X[:, 1], c=kmeans.labels_)
ax. set_title ("K-Means Clustering Results with K=2");

plt.grid(True)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3,
                random_state=3,
                n_init=20).fit(X)

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(X[:, 0], X[:, 1], c=kmeans.labels_)
ax. set_title ("K-Means Clustering Results with K=3");

plt.grid(True)
plt.show()

In [ ]:
kmeans1 = KMeans(n_clusters=3,
                 random_state=3,
                 n_init =1).fit(X)

kmeans20 = KMeans(n_clusters=3,
                  random_state=3,
                  n_init =20).fit(X);

kmeans1.inertia_, kmeans20.inertia_

#### Hierarchical Clustering

In [ ]:
HClust = AgglomerativeClustering

hc_comp = HClust(distance_threshold=0,
                 n_clusters=None,
                 linkage='complete')

hc_comp.fit(X)

In [ ]:
hc_avg = HClust(distance_threshold=0,
                n_clusters=None,
                linkage='average');

hc_avg.fit(X)
hc_sing = HClust(distance_threshold=0,
                 n_clusters=None,
                 linkage='single');

hc_sing.fit(X);

In [ ]:
D = np.zeros ((X.shape[0], X.shape[0]));

for i in range(X.shape[0]):
    x_ = np. multiply.outer(np.ones(X.shape[0]), X[i])
    D[i] = np.sqrt(np.sum((X - x_)**2, 1));
    hc_sing_pre = HClust(distance_threshold=0,
                         n_clusters =None,
                         metric='precomputed',
                         linkage='single')
hc_sing_pre .fit(D)

In [ ]:
cargs = {'color_threshold':-np.inf,
         'above_threshold_color':'black'}

linkage_comp = compute_linkage(hc_comp)
fig, ax = plt.subplots (1, 1, figsize=(8, 8))
dendrogram (linkage_comp,
            ax=ax,
            **cargs);

In [ ]:
fig, ax = plt. subplots(1, 1, figsize=(8, 8))

dendrogram (linkage_comp,
            ax=ax,
            color_threshold=4,
            above_threshold_color ='black');

In [ ]:
cut_tree(linkage_comp, n_clusters=4).T

In [ ]:
cut_tree(linkage_comp, height=5)

In [ ]:
scaler = StandardScaler()

X_scale = scaler.fit_transform(X)

hc_comp_scale = HClust(distance_threshold=0,
                       n_clusters=None,
                       linkage='complete').fit(X_scale)

linkage_comp_scale = compute_linkage(hc_comp_scale)
fig, ax = plt. subplots(1, 1, figsize=(8, 8))
dendrogram(linkage_comp_scale, ax=ax, ** cargs)
ax.set_title(" Hierarchical Clustering with Scaled Features");

In [ ]:
X = np.random.standard_normal((30, 3))

corD = 1 - np.corrcoef(X)

hc_cor = HClust(linkage='complete',
                distance_threshold=0,
                n_clusters=None,
                metric='precomputed')

hc_cor.fit(corD)
linkage_cor = compute_linkage(hc_cor)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
dendrogram (linkage_cor, ax=ax, **cargs)
ax. set_title ("Complete Linkage with Correlation -Based Dissimilarity");